In [1]:
import pandas as pd
import numpy as np

In [46]:
def str_clean(x):
    if x=="":
        return False
    else:
        return True
    
def admin2_name(city):
    global admin2_frame
    state=str(city.loc["State"])
    admin2_code=str(city.loc["Admin2"])
    try:
        city["Admin2 Name"]=admin2_frame.query("State==@state").query("Admin2Code==@admin2_code").index[0]
        
    except IndexError:
        city[:]=pd.NaT
    return city

city_frame=pd.DataFrame()
with open('cities500.txt','r',encoding='utf-8') as f:
    for line in f:
        data=list(filter(str_clean, line[:-1].split('\t')))
        try:
            str(int(data[3]));
        except ValueError:
            data.pop(3);
        if data[7]=="US" and data[6]=="PPL":
            if len(data)>15:
                data.pop(10);
            data.insert(10,"")
            city_frame=city_frame.append(pd.DataFrame(data).T)
        else:
            continue

city_frame=(
    city_frame.drop([0,2,5,6,7,12],axis=1)
    .rename(index=str,columns={1:"City (US)",3:"Latitude",4:"Longitude",10:"Admin2 Name",
                                                8:"State",9:"Admin2",11:"Population",
                                                13: "Elevation (m)", 14:"Timezone",15:"Date Updated"})
    .set_index("City (US)")
    .sort_values(by=["State","Admin2"])
)

admin2_frame=pd.DataFrame()
with open("admin2Codes.txt",'r',encoding='utf-8') as read:
    for line in read:
        data=list(filter(str_clean, line[:-1].replace('.','\t').split('\t')))
        if data[0].__contains__("US"):
            admin2_frame=admin2_frame.append(pd.DataFrame(data[:4]).T)
        else:
            continue
         
admin2_frame=(
    admin2_frame.drop([0],axis=1)     
    .rename(index=str,columns={1:"State",2:"Admin2Code",3:"Division Name"})
    .set_index("Division Name")
)

city_frame=city_frame.apply(admin2_name,axis=1)
city_frame.dropna(how='all',inplace=True)

city_frame.to_csv("citiesUS.csv")